## KLT TRACKING FOR MULTIPLE OBJECTS IN A GIVEN SCENE

1. SETTING UP VIDEO CAPTURE AND PREPROCESSING FRAMES

In [117]:
import cv2
import numpy as np

# Capture video from file or webcam (replace 'video.mp4' with your video file path)
cap = cv2.VideoCapture('traffic3.mp4')

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Parameters for Lucas-Kanade optical flow (KLT)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the first frame
ret, old_frame = cap.read()
if not ret:
    print("Cannot read video file")
    cap.release()
    exit()

# Convert the first frame to grayscale
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)


In [118]:

# Define train-test split ratio
train_ratio = 0.8

# Determine total frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
train_frames = int(total_frames * train_ratio)

# Reset the video capture to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)


True

 2.DETECTING FEATURES IN INITIAL FRAME

In [130]:
import cv2
import numpy as np

# Parameters for ShiTomasi corner detection (good features to track)
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Load the video
cap = cv2.VideoCapture('traffic2.mp4')

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Read the first frame
ret, frame = cap.read()
if not ret:
    print("Error: Could not read the video frame.")
    cap.release()
    exit()

# Convert frame to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect features (key points) to track
features = cv2.goodFeaturesToTrack(gray, mask=None, **feature_params)

# Check if points are detected
if features is None:
    print("No features detected.")
    cap.release()
    exit()

# Draw the detected features as circles
for point in features:
    x, y = point.ravel()
    cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Green circles for features

# Display the frame with features
cv2.imshow('Feature Detection', frame)

# Wait for key press and close window
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()




In [120]:

# List of video files to process
video_files = ['Easy.mp4', 'traffic2.mp4', 'traffic3.mp4']

for video_file in video_files:
    print(f"Processing video: {video_file}")
    
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_file}.")
        continue

    # Calculate train-test split
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    train_frames = int(total_frames * train_ratio)

    # Read the first frame for initial feature detection
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, first_frame = cap.read()
    if not ret:
        print(f"Error: Could not read the first frame of {video_file}.")
        cap.release()
        continue

    # Convert the first frame to grayscale for feature detection
    first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    features = cv2.goodFeaturesToTrack(first_gray, mask=None, **feature_params)

    # Process frames for training segment
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for i in range(train_frames):
        ret, frame = cap.read()
        if not ret:
            print(f"Reached end of training segment for {video_file}.")
            break

        # Insert tracking and feature update code here for training

    # Process frames for testing segment
    for i in range(train_frames, total_frames):
        ret, frame = cap.read()
        if not ret:
            print(f"Reached end of testing segment for {video_file}.")
            break

        # Insert tracking and feature update code here for testing

    cap.release()
    print(f"Completed processing video: {video_file}")


Processing video: Easy.mp4
Completed processing video: Easy.mp4
Processing video: traffic2.mp4
Completed processing video: traffic2.mp4
Processing video: traffic3.mp4
Completed processing video: traffic3.mp4


3. TRACKING KEY POINTS ACROSS FRAMES

In [135]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture("traffic4.mp4")  # Replace with your own video file

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Parameters for optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize the first frame
ret, frame = cap.read()
if not ret:
    print("Error: Failed to read first frame.")
    exit()

# Convert to grayscale for optical flow
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect good features to track in the first frame
initial_points = cv2.goodFeaturesToTrack(old_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7))

# Initialize a mask for drawing the tracks
mask = np.zeros_like(frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for optical flow
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow to track points in the new frame
    new_points, status, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, initial_points, None, **lk_params)

    # Select good points (points that were successfully tracked)
    good_new = new_points[status == 1]
    good_old = initial_points[status == 1]

    # Re-detect points if necessary (when many points are lost)
    if len(good_new) < 10:  # If fewer than 10 points are left, re-detect features
        initial_points = cv2.goodFeaturesToTrack(frame_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7))
        good_new = initial_points
        good_old = good_new

    # Draw tracking lines and points on the frame
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = int(new[0]), int(new[1])  # Convert to integers
        c, d = int(old[0]), int(old[1])  # Convert to integers
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)  # Green line for tracking
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)  # Red circle for new points

    # Overlay the mask on the frame
    img = cv2.add(frame, mask)

    # Display the tracking result
    cv2.imshow("Tracking", img)

    # Update the previous frame and points
    old_gray = frame_gray.copy()
    initial_points = good_new.reshape(-1, 1, 2)

    # Exit on 'q' key
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



4. DETECTING OBJECTS USING BACKGROUND SUBTRACTION

I. SETTING UP BACKGROUND SUBTRACTOR

In [133]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture("traffic4.mp4")  # Replace with your own video file

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Parameters for KLT Tracker
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize tracking points list
tracking_points = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction to detect moving objects
    fg_mask = bg_subtractor.apply(frame)
    
    # Threshold the mask to binary
    _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)

    # Find contours of moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size to avoid noise
    bounding_boxes = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(cnt)
            bounding_boxes.append((x, y, w, h))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box

            # Initialize tracking points at the center of each bounding box
            center = np.array([[x + w / 2, y + h / 2]], dtype=np.float32)
            tracking_points.append(center)

    cv2.imshow("Detection", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


II.TRACKING THE DETECTED OBJECTS



In [123]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture("traffic3.mp4")  # Replace with your own video file

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Parameters for KLT Tracker
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize tracking points and paths
tracking_points = []
tracking_paths = []

# Read the first frame and initialize prev_gray
ret, frame = cap.read()
if not ret:
    print("Error: Could not read video.")
    cap.release()
    cv2.destroyAllWindows()
else:
    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale for optical flow
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction to detect moving objects
    fg_mask = bg_subtractor.apply(frame)
    
    # Threshold the mask to binary
    _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)

    # Find contours of moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size to avoid noise
    bounding_boxes = []
    new_tracking_points = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(cnt)
            bounding_boxes.append((x, y, w, h))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box in green

            # Initialize tracking points at the center of each bounding box
            center = np.array([[x + w / 2, y + h / 2]], dtype=np.float32)
            new_tracking_points.append(center)

    # Update tracking paths
    if len(tracking_points) > 0:
        # Calculate optical flow
        new_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray_frame, tracking_points, None, **lk_params)
        
        # Draw lines for each tracked path
        updated_tracking_paths = []
        updated_tracking_points = []
        for i, (new, old) in enumerate(zip(new_points, tracking_points)):
            if status[i]:  # Only draw if the point was successfully tracked
                a, b = new.ravel()
                c, d = old.ravel()

                # Only add a line segment every few frames to avoid dense trails
                if len(tracking_paths[i]) == 0 or np.linalg.norm(np.array(tracking_paths[i][-1]) - np.array((a, b))) > 5:
                    tracking_paths[i].append((a, b))
                
                # Prune paths to the last few points to keep them minimal
                if len(tracking_paths[i]) > 2:
                    tracking_paths[i] = tracking_paths[i][-2:]

                # Draw the latest path segment
                if len(tracking_paths[i]) == 2:
                    pt1 = tuple(map(int, tracking_paths[i][0]))
                    pt2 = tuple(map(int, tracking_paths[i][1]))
                    cv2.line(frame, pt1, pt2, (255, 0, 255), 2)  # Draw tracking lines in purple
                
                # Keep only active points and paths
                updated_tracking_paths.append(tracking_paths[i])
                updated_tracking_points.append(new)

        # Update tracking points and paths with only active ones
        tracking_points = np.array(updated_tracking_points, dtype=np.float32).reshape(-1, 1, 2)
        tracking_paths = updated_tracking_paths
    else:
        # Initialize tracking points and paths
        tracking_points = np.array(new_tracking_points, dtype=np.float32)
        tracking_paths = [[(p[0][0], p[0][1])] for p in tracking_points]

    # Add new tracking points if they were not already being tracked
    new_tracking_points = np.array(new_tracking_points, dtype=np.float32)
    if len(new_tracking_points) > 0:
        tracking_points = np.concatenate((tracking_points, new_tracking_points), axis=0)
        tracking_paths.extend([[(p[0][0], p[0][1])] for p in new_tracking_points])

    # Update the previous frame and tracking points
    prev_gray = gray_frame.copy()
    cv2.imshow("Detection and Tracking", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()





III. IMPROVING THE TRACKER (FINE TUNING)

In [129]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture("traffic3.mp4")  # Replace with your video path

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Parameters for KLT Tracker
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize tracking points and paths
tracking_points = np.array([], dtype=np.float32)
tracking_paths = []  # Paths to store movement history

# Set the maximum number of lines (cars) to track
MAX_TRACKING_LINES = 10  # Track more cars
MAX_PATH_LENGTH = 50  # Increase line length

# Read the first frame and initialize prev_gray
ret, frame = cap.read()
if not ret:
    print("Error: Could not read video.")
    cap.release()
    cv2.destroyAllWindows()
else:
    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale for optical flow
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction to detect moving objects
    fg_mask = bg_subtractor.apply(frame)

    # Threshold the mask to binary
    _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)

    # Find contours of moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on size to avoid noise
    new_tracking_points = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(cnt)
            center = np.array([[x + w / 2, y + h / 2]], dtype=np.float32)  # Center of bounding box
            new_tracking_points.append(center)

            # Draw bounding box around the detected car
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box

    # Update tracking points and paths if there are points
    if len(tracking_points) > 0:
        # Calculate optical flow
        new_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray_frame, tracking_points, None, **lk_params)

        # Update paths and draw tracking lines
        updated_paths = []
        updated_tracking_points = []
        for i, (new, old) in enumerate(zip(new_points, tracking_points)):
            if status[i]:
                # Append new point to the existing path
                tracking_paths[i].append((new[0][0], new[0][1]))
                updated_paths.append(tracking_paths[i])
                updated_tracking_points.append(new)

                # Draw the tracking line for each car
                for j in range(1, len(tracking_paths[i])):
                    pt1 = (int(tracking_paths[i][j - 1][0]), int(tracking_paths[i][j - 1][1]))
                    pt2 = (int(tracking_paths[i][j][0]), int(tracking_paths[i][j][1]))
                    cv2.line(frame, pt1, pt2, (0, 0, 255), 2)  # Red line for tracking

                # Limit path length for clarity
                if len(tracking_paths[i]) > MAX_PATH_LENGTH:
                    tracking_paths[i].pop(0)  # Remove the oldest point

        tracking_paths = updated_paths  # Update paths with only active ones
        tracking_points = np.array(updated_tracking_points, dtype=np.float32)  # Update active points

    # Initialize new tracking points if no prior points exist
    if len(tracking_points) == 0 and len(new_tracking_points) > 0:
        tracking_points = np.array(new_tracking_points, dtype=np.float32)
        tracking_paths = [[(p[0][0], p[0][1])] for p in new_tracking_points]

    # Add new tracking points to current ones
    elif len(new_tracking_points) > 0:
        tracking_points = np.concatenate((tracking_points, new_tracking_points), axis=0)
        tracking_paths.extend([[(p[0][0], p[0][1])] for p in new_tracking_points])

    # Limit the number of tracking lines (cars) to MAX_TRACKING_LINES
    if len(tracking_paths) > MAX_TRACKING_LINES:
        tracking_paths = tracking_paths[:MAX_TRACKING_LINES]
        tracking_points = tracking_points[:MAX_TRACKING_LINES]

    # Update previous frame for optical flow
    prev_gray = gray_frame.copy()

    # Show the output
    cv2.imshow("Detection and Tracking", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()










5. CALCULATING THE ACCURACY

In [125]:
import cv2
import numpy as np

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2  # Corrected unpacking of box2

    # Calculate the intersection coordinates
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    # Calculate the area of the intersection rectangle
    intersection_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate the union area
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area if union_area != 0 else 0
    return iou

# Example detected and ground truth boxes (replace with actual data)
detected_boxes = [
    [100, 150, 80, 60],
    [200, 250, 90, 70]
]

ground_truth_boxes = [
    [105, 155, 85, 65],
    [210, 260, 85, 65]
]

# Initialize counters
correct_tracks = 0
matched_gt_boxes = set()

# Loop through detected and ground truth boxes
for det_idx, det_box in enumerate(detected_boxes):
    for gt_idx, gt_box in enumerate(ground_truth_boxes):
        # Calculate IoU between detected and ground truth boxes
        iou = calculate_iou(det_box, gt_box)
        print(f"Detected box {det_idx} and Ground Truth box {gt_idx} IoU: {iou:.2f}")

        # Check if IoU is above threshold and ground truth box has not been matched yet
        if iou > 0.5 and gt_idx not in matched_gt_boxes:
            correct_tracks += 1
            matched_gt_boxes.add(gt_idx)  # Mark this GT box as matched
            print(f"Match found: Detected box {det_idx} with Ground Truth box {gt_idx}")
            break  # Stop checking this detected box once matched

# Calculate total ground truth boxes and accuracy
total_ground_truth = len(ground_truth_boxes)
accuracy = correct_tracks / total_ground_truth if total_ground_truth > 0 else 0

print(f"\nTracking accuracy: {accuracy:.2f}")


Detected box 0 and Ground Truth box 0 IoU: 0.67
Match found: Detected box 0 with Ground Truth box 0
Detected box 1 and Ground Truth box 0 IoU: 0.00
Detected box 1 and Ground Truth box 1 IoU: 0.68
Match found: Detected box 1 with Ground Truth box 1

Tracking accuracy: 1.00


6. VISUALIZING THE ACCURACY OF THE TRACKER WITH THE RESPECTED VIDEO FOR OBJECT DETECTION

In [126]:
import cv2
import numpy as np

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate the intersection coordinates
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    # Calculate the area of the intersection rectangle
    intersection_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate the union area
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area if union_area != 0 else 0
    return iou

# Example image (replace with an actual image)
image = np.ones((500, 500, 3), dtype=np.uint8) * 255  # White background

# Example detected and ground truth boxes
detected_boxes = [
    [100, 150, 80, 60],
    [200, 250, 90, 70]
]

ground_truth_boxes = [
    [105, 155, 85, 65],
    [210, 260, 85, 65]
]

# Set IoU threshold
iou_threshold = 0.5

# Initialize counters
correct_tracks = 0
matched_gt_boxes = set()

# Loop through detected and ground truth boxes
for det_idx, det_box in enumerate(detected_boxes):
    for gt_idx, gt_box in enumerate(ground_truth_boxes):
        iou = calculate_iou(det_box, gt_box)

        # Draw the ground truth boxes (green) and detected boxes (blue)
        cv2.rectangle(image, (gt_box[0], gt_box[1]), (gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]), (0, 255, 0), 2)
        cv2.rectangle(image, (det_box[0], det_box[1]), (det_box[0] + det_box[2], det_box[1] + det_box[3]), (255, 0, 0), 2)

        # If IoU is above threshold and the GT box is not matched
        if iou > iou_threshold and gt_idx not in matched_gt_boxes:
            correct_tracks += 1
            matched_gt_boxes.add(gt_idx)  # Mark this GT box as matched

            # Draw a label indicating the match
            label = f"Match {iou:.2f}"
            cv2.putText(image, label, (det_box[0], det_box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Calculate accuracy
accuracy = correct_tracks / len(ground_truth_boxes) if len(ground_truth_boxes) > 0 else 0

# Show the image with boxes and IoU labels
cv2.imshow("IoU Visualization", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print the tracking accuracy
print(f"Tracking accuracy: {accuracy:.2f}")



Tracking accuracy: 1.00


5. TESTING KLT FOR FACIAL RECOGNITION

In [127]:
'''import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Termination criteria for the KLT tracker (Lucas-Kanade Optical Flow)
lk_params = dict(winSize=(21, 21), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Variables for face tracking
face_roi = None
tracked_points = None
old_gray = None
p0 = None

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If a face is detected
    if len(faces) > 0:
        # Take the first detected face (you can add more logic to choose a specific one if needed)
        x, y, w, h = faces[0]
        face_roi = frame[y:y+h, x:x+w]

        # Make the face bounding box more prominent by increasing thickness and using a different color
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)  # Green color with 3px thickness

        # Dynamically select feature points around the face using the detected face region
        step_size = 10  # Adjust step size to select more refined feature points
        feature_points = []
        for i in range(0, w, step_size):
            for j in range(0, h, step_size):
                feature_points.append((x + i, y + j))  # Feature points within the face region

        # Convert feature points to numpy array for tracking
        tracked_points = np.array(feature_points, dtype=np.float32)

        # Initialize the optical flow with the detected feature points if it's the first frame
        if old_gray is None and tracked_points is not None:
            old_gray = gray
            p0 = tracked_points.reshape(-1, 1, 2)

    if tracked_points is not None and old_gray is not None:
        # Track the features using optical flow (Lucas-Kanade method)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

        # Ensure that optical flow was successfully computed (p1 is not None)
        if p1 is not None:
            # Select good points (tracked correctly)
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Draw the tracked points (as small plus signs) inside the bounding box
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                # Ensure that point is inside the face region
                if x <= a <= x + w and y <= b <= y + h:
                    # Draw the white plus sign with improved visibility and sharpness
                    cv2.line(frame, (int(a) - 6, int(b)), (int(a) + 6, int(b)), (255, 255, 255), 2)  # Horizontal line
                    cv2.line(frame, (int(a), int(b) - 6), (int(a), int(b) + 6), (255, 255, 255), 2)  # Vertical line

            # Update the previous frame and previous points for the next iteration
            old_gray = gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

    # Display the frame with face detection and feature tracking
    cv2.imshow('Face Detection and Feature Tracking', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
'''

"import cv2\nimport numpy as np\n\n# Initialize the webcam\ncap = cv2.VideoCapture(0)\n\n# Load the pre-trained Haar Cascade Classifier for face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')\n\n# Termination criteria for the KLT tracker (Lucas-Kanade Optical Flow)\nlk_params = dict(winSize=(21, 21), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))\n\n# Variables for face tracking\nface_roi = None\ntracked_points = None\nold_gray = None\np0 = None\n\nwhile True:\n    # Read a frame from the webcam\n    ret, frame = cap.read()\n    if not ret:\n        break\n\n    # Convert the frame to grayscale\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    # Detect faces in the frame\n    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))\n\n    # If a face is detected\n    if len(faces) > 0:\n        # Take the first detected face (you can add more

6.TRACKING THE FEATURES USING KLT


In [128]:
'''import cv2
import numpy as np

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Termination criteria for the KLT tracker (Lucas-Kanade Optical Flow)
lk_params = dict(winSize=(21, 21), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Variables for face tracking
face_roi = None
tracked_points = None
old_gray = None
p0 = None
tracked_points_history = []  # To store previous points for drawing tracking lines

# Define colors for different features (eyes, nose, lips, jaw)
eye_color = (255, 0, 0)  # Blue for eyes
nose_color = (0, 255, 0)  # Green for nose
lip_color = (0, 0, 255)  # Red for lips
jaw_color = (255, 255, 0)  # Cyan for jaw

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If a face is detected
    if len(faces) > 0:
        # Take the first detected face (you can add more logic to choose a specific one if needed)
        x, y, w, h = faces[0]
        face_roi = frame[y:y+h, x:x+w]

        # Make the face bounding box more prominent (black color with thicker line)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 3)  # Black color with 3px thickness

        # Dynamically select fewer feature points around the face using the detected face region
        step_size = 30  # Increased step size for fewer feature points
        feature_points = []
        for i in range(0, w, step_size):
            for j in range(0, h, step_size):
                feature_points.append((x + i, y + j))  # Feature points within the face region

        # Convert feature points to numpy array for tracking
        tracked_points = np.array(feature_points, dtype=np.float32)

        # Initialize the optical flow with the detected feature points if it's the first frame
        if old_gray is None and tracked_points is not None:
            old_gray = gray
            p0 = tracked_points.reshape(-1, 1, 2)

    if tracked_points is not None and old_gray is not None:
        # Track the features using optical flow (Lucas-Kanade method)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

        # Ensure that optical flow was successfully computed (p1 is not None)
        if p1 is not None:
            # Select good points (tracked correctly)
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Color-code the tracked points based on the feature category
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                # Ensure that point is inside the face region
                if x <= a <= x + w and y <= b <= y + h:
                    # Draw colored points (different colors for different features)
                    if i % 4 == 0:  # Eyes (blue)
                        color = eye_color
                    elif i % 4 == 1:  # Nose (green)
                        color = nose_color
                    elif i % 4 == 2:  # Lips (red)
                        color = lip_color
                    else:  # Jaw (cyan)
                        color = jaw_color
                    
                    # Draw the feature points (as small plus signs) with the corresponding color
                    cv2.line(frame, (int(a) - 8, int(b)), (int(a) + 8, int(b)), color, 3)  # Horizontal line
                    cv2.line(frame, (int(a), int(b) - 8), (int(a), int(b) + 8), color, 3)  # Vertical line

                    # Track movement with lines (previous position to current)
                    if len(tracked_points_history) > 0:
                        prev_points = tracked_points_history[-1]
                        for j, prev in enumerate(prev_points):
                            if j < len(good_new):  # Ensure no index out of bounds
                                # Draw thicker and smoother lines between old and new positions
                                cv2.line(frame, (int(prev[0]), int(prev[1])), (int(good_new[j][0]), int(good_new[j][1])), color, 4)  # Smoother and bigger lines

            # Update the tracked points history for drawing movement lines
            tracked_points_history.append(good_new)
            if len(tracked_points_history) > 15:  # Limit history to last 15 frames for smoother lines
                tracked_points_history.pop(0)

            # Update the previous frame and previous points for the next iteration
            old_gray = gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

    # Display the frame with face detection, feature tracking, and color-coded features
    cv2.imshow('Face Detection and Movement Tracking', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
'''

"import cv2\nimport numpy as np\n\n# Initialize the webcam\ncap = cv2.VideoCapture(0)\n\n# Load the pre-trained Haar Cascade Classifier for face detection\nface_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')\n\n# Termination criteria for the KLT tracker (Lucas-Kanade Optical Flow)\nlk_params = dict(winSize=(21, 21), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))\n\n# Variables for face tracking\nface_roi = None\ntracked_points = None\nold_gray = None\np0 = None\ntracked_points_history = []  # To store previous points for drawing tracking lines\n\n# Define colors for different features (eyes, nose, lips, jaw)\neye_color = (255, 0, 0)  # Blue for eyes\nnose_color = (0, 255, 0)  # Green for nose\nlip_color = (0, 0, 255)  # Red for lips\njaw_color = (255, 255, 0)  # Cyan for jaw\n\nwhile True:\n    # Read a frame from the webcam\n    ret, frame = cap.read()\n    if not ret:\n        break\n\n    # Convert th